In [ ]:
mp# libraries importing
import pandas as pd
import base64
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import plotly.express as px

from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler


In [ ]:
# Data reading

rfm=pd.read_csv('/content/reorder second - Copy.csv')
rfm.drop_duplicates(inplace=True)

rfm =rfm.fillna(0)

In [ ]:
# Variables creation

mean = '_mean'
std = '_std'
column_name = ['RETAILER_ID','district_id']

fraction1 = 1/4
fraction2 = 1/3
fraction3 = 1/2

conditions = [True,True]

In [ ]:

# Auto Variable Creations in a dict
dict_for_mean_t1 = {}
dict_for_std_t1 = {}

dict_for_mean_t2 = {}
dict_for_std_t2 = {}

dict_for_mean_t3 = {}
dict_for_std_t3 = {}

for i in range(len(rfm.select_dtypes(['float','int']).columns)):
  if any(column_names == rfm.select_dtypes(['float','int']).columns[i] for column_names in column_name):
    continue
  else:
    key1 = str(rfm.select_dtypes(['float','int']).columns[i]+mean)
    key2 = str(rfm.select_dtypes(['float','int']).columns[i]+std)

    dict_for_mean_t1[key1]=1
    dict_for_std_t1[key2]=1

    dict_for_mean_t2[key1]=1
    dict_for_std_t2[key2]=1

    dict_for_mean_t3[key1]=1
    dict_for_std_t3[key2]=1


In [ ]:
dict_for_std_t3

In [ ]:
# automation functions

# Spliting
def treat_control():
  total = rfm.copy()
  # Dataframe creation
  df_control=pd.DataFrame(columns=total.columns)
  df_treatment1=pd.DataFrame(columns=total.columns)
  df_treatment2=pd.DataFrame(columns=total.columns)
  df_treatment3=pd.DataFrame(columns=total.columns)

  # Groups split
  df_control = total.sample(frac=fraction1)
  # df_treatment1 = total.drop(df_control.index)
  df_treatment1 = total.drop(df_control.index).sample(frac=fraction2)

  df_treatment2 = pd.concat([df_treatment2, total.drop(df_control.index).drop(df_treatment1.index)]).sample(frac=fraction3)
  df_treatment3 = pd.concat([df_treatment3, total.drop(df_control.index).drop(df_treatment1.index).drop(df_treatment2.index)])
  return df_control, df_treatment1, df_treatment2, df_treatment3

# balancing calculation
def calculation(dict_for_mean_t1 , dict_for_std_t1, dict_for_mean_t2 , dict_for_std_t2, dict_for_mean_t3 , dict_for_std_t3):
  global control, treat, treat2, treat3
  control, treat,treat2,treat3=treat_control()

  # T1 calculation
  for key, value in dict_for_mean_t1.items():
    dict_for_mean_t1[key]=np.abs((treat\
     [key.rstrip(mean)].mean() - control\
      [key.rstrip(mean)].mean()) / control\
       [key.rstrip(mean)].mean())

  for key, value in dict_for_std_t1.items():
    dict_for_std_t1[key]=np.abs((treat\
     [key.rstrip(std)].std() - control\
      [key.rstrip(std)].std()) / control\
       [key.rstrip(std)].std())

  # T2 calculation
  for key, value in dict_for_mean_t2.items():
    dict_for_mean_t2[key]=np.abs((treat2\
     [key.rstrip(mean)].mean() - control\
      [key.rstrip(mean)].mean()) / control\
       [key.rstrip(mean)].mean())

  for key, value in dict_for_std_t2.items():
    dict_for_std_t2[key]=np.abs((treat2\
     [key.rstrip(std)].std() - control\
      [key.rstrip(std)].std()) / control\
       [key.rstrip(std)].std())

  #T3 calulation
  for key, value in dict_for_mean_t3.items():
    dict_for_mean_t3[key]=np.abs((treat3\
     [key.rstrip(mean)].mean() - control\
      [key.rstrip(mean)].mean()) / control\
       [key.rstrip(mean)].mean())

  for key, value in dict_for_std_t3.items():
    dict_for_std_t3[key]=np.abs((treat3\
     [key.rstrip(std)].std() - control\
      [key.rstrip(std)].std()) / control\
       [key.rstrip(std)].std())

  return dict_for_mean_t1 , dict_for_std_t1, dict_for_mean_t2 , dict_for_std_t2, dict_for_mean_t3 , dict_for_std_t3 , control, treat,treat2,treat3

# Merge the results for only 1 treatment group
def Merge_2(std_dict, mean_dict):
    res = {**std_dict, **mean_dict}
    return res

# Merge the results for more than 1 treatment
def Merge(dict_for_std_t1, dict_for_mean_t1):
  result = {}

  for key in set(dict_for_std_t1) | set(dict_for_mean_t1):
    if key in dict_for_std_t1 and key in dict_for_mean_t1:
      result[key+'_1'] = dict_for_std_t1[key]
      result[key+'_2'] = dict_for_mean_t1[key]
    elif key in dict_for_std_t1:
      result[key+'_1'] = dict_for_std_t1[key]
    else:
      result[key+'_2'] = dict_for_mean_t1[key]

  return result

In [ ]:
# Balancing loop

while any(conditions):

  # calculating
  mean_dict_t1 , std_dict_t1, mean_dict_t2 , std_dict_t2, mean_dict_t3 , std_dict_t3,contorl, treat, treat2, treat3 = calculation(dict_for_mean_t1 , dict_for_std_t1,dict_for_mean_t2 , dict_for_std_t2, dict_for_mean_t3 , dict_for_std_t3)

  # Results collecting
    # T1
  total_t1 = Merge(mean_dict_t1, std_dict_t1)
    # T2
  total_t2 = Merge(mean_dict_t2, std_dict_t2)
    # T3
  total_t3 = Merge(mean_dict_t3, std_dict_t3)
    # Merge results
  total_1 = Merge(total_t1, total_t2)
  total_2 = Merge(total_1, total_t3)
    # Coniditons check
  conditions = [value > 0.05 for value in total_2.values()]

In [ ]:
total_2

In [ ]:
# Add the tag
control['tag']='control'
treat['tag']='treat'
treat2['tag']='treat2'
treat3['tag']='treat3'

# append the lists
low = control.append(treat, ignore_index=True)
low = low.append(treat2, ignore_index=True)
low = low.append(treat3, ignore_index=True)

# save the data
low.to_csv('Reorder second release.csv')

<ipython-input-138-c050a6889698>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  low=df_control.append(df_treatment1, ignore_index=True)
